Heroes of Pymoli Data Analysis
---------------------------------------------
Of the 576 total number of unique players identified in the 780 purhcases, 84% were male, 14% were female, remaining were not disclosed.  This indicates that males are the main gender purhcasing games, but there are probably opportunities to gain in the female segment.

The 20-24 age bracket consisted of 45% of the purchasers, with the 15-19 age bracket second at 19% of the sample.  Percentages begin declining beyond the age of 29.

The most popular game (Oathbreaker, Last Hope of the Breaking Storm	) is also the most profitable item.  Further investigation of purhcase breakdown and marketing of this title, would be beneficial for new products.  




In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data_df = pd.read_csv(file_to_load)

#see a sample of the data
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [2]:
#to find the number of unique players 
num_unique_players = len(purchase_data_df["SN"].unique())
#print(num_unique_players)

#looks like from the the example they put it in a dataframe..so will follow the output guidelines
pd.DataFrame({"Total Number of Players": [num_unique_players]})

,Total Number of Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
#Purchasing Analysis (Total)

#compute items 
num_unique_items = len(purchase_data_df["Item ID"].unique())
avg_price = purchase_data_df["Price"].mean()
num_purchases = len(purchase_data_df)
tot_revenue = purchase_data_df["Price"].sum()

#put these things in a summary table
summary_table_df = pd.DataFrame({"Number of Unique Items": [num_unique_items],
                                 "Average Price": [avg_price],
                                 "Number of Purchases": [num_purchases],
                                 "Total Revenue": [tot_revenue]})
#summary_table_df.dtypes
#convert to proper data types
summary_table_df["Average Price"] = summary_table_df["Average Price"].astype(float).map("${:,.2f}".format)
summary_table_df["Total Revenue"] = summary_table_df["Total Revenue"].astype(float).map("${:,.2f}".format)

summary_table_df.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
#Gender Demographics

#group this df by gender..but make sure people are not counted twice..reset the index, too <last part important!!>
grouped_by_gender_df = purchase_data_df.groupby("Gender")["SN"].nunique().reset_index()

#add % of player to the df
grouped_by_gender_df["Percentage of Players"] = 100 * grouped_by_gender_df["SN"]/grouped_by_gender_df["SN"].sum()

#make a summary table
gender_summary_df = grouped_by_gender_df

# format the % column
gender_summary_df['Percentage of Players'] = gender_summary_df['Percentage of Players'].map("{:,.2f}".format)

#sort by percentage of players
gender_summary_df = gender_summary_df.sort_values(['Percentage of Players'],ascending = False)

#show the df
gender_summary_df

,Gender,SN,Percentage of Players
1,Male,484,84.03
0,Female,81,14.06
2,Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
#Purchasing Analysis (by Gender)
#need purchase count, avg purchase price, avg purch total/per person   BY GENDER

purch_count_by_gender = purchase_data_df.groupby(["Gender"]).count()["Price"].rename("Purchase Count")
avg_puch_price_by_gender = purchase_data_df.groupby(["Gender"]).mean()["Price"].rename("Avg Value")
tot_purch_by_gender = purchase_data_df.groupby(["Gender"]).sum()["Price"].rename("Purch Value")

#find avg purch by person (already grouped by gender)
num_unique_people = purchase_data_df.groupby("Gender")["SN"].nunique()
avg_purch_tot_per_person = tot_purch_by_gender/num_unique_people


# Place all of the data found into a summary DataFrame
summary_table = pd.DataFrame({"Purchase Count": purch_count_by_gender,
                              "Average Purchase Price": avg_puch_price_by_gender,
                              "Total Purchase Value": tot_purch_by_gender,
                              "Avg Total Purchase Per Person": avg_purch_tot_per_person})

#clean up $ amounts
summary_table["Average Purchase Price"]  = summary_table["Average Purchase Price"].map("${:.2f}".format)
summary_table["Total Purchase Value"] = summary_table["Total Purchase Value"].map("${:.2f}".format)
summary_table["Avg Total Purchase Per Person"] = summary_table["Avg Total Purchase Per Person"].map("${:.2f}".format)

#print it out
summary_table

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
#Age Demographics

#b4 binning, lets look at min and max of ages
age_min = purchase_data_df["Age"].min()
age_max = purchase_data_df["Age"].max()
#print(age_min, age_max)

# lets bin under 10 for 1 bin, and 7 more
bins = [0, 9, 14, 19, 24, 29, 34, 39, 44]
age_ranges_labels = ["<10","10-14","15-19", "20-24", "25-29", "30-34", "35-39", ">40"]

# slice the data and add a bin column for the bins on age groups
purchase_data_df["Age Group"] = pd.cut(purchase_data_df["Age"], bins, labels=age_ranges_labels)
purchase_data_df.head(10)

# Create a GroupBy object based upon "Age Group" and put in object
purch_data_age_gp_df = purchase_data_df.groupby("Age Group")["SN"].nunique().reset_index()
purch_data_age_gp_df['Percentage of Players'] = 100 * purch_data_age_gp_df['SN']/purch_data_age_gp_df['SN'].sum()
age_summary = purch_data_age_gp_df[["Age Group", "SN", "Percentage of Players"]].sort_values(['Age Group'])
age_summary = age_summary.reset_index(drop=True)

# format columns
age_summary['Percentage of Players'] = age_summary['Percentage of Players'].map("{:,.2f}".format)

#now have Age Group as the index
age_summary = age_summary.set_index('Age Group')
age_summary = age_summary.rename(columns = {'SN': 'Total Count'}) 
age_summary

,Total Count,Percentage of Players
Age Group,,
<10,17,2.96
10-14,22,3.83
15-19,107,18.61
20-24,258,44.87
25-29,77,13.39
30-34,52,9.04
35-39,31,5.39
>40,11,1.91


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# Figure out the minimum and maximum ages for purchasing
age_min = purchase_data_df["Age"].min()
age_max = purchase_data_df["Age"].max()
#print(f"Max = {age_max} and Min = {age_min}")

In [8]:
# lets bin under 10 for 1 bin, and 7 more
bins = [0, 9, 14, 19, 24, 29, 34, 39, 44]
age_ranges_labels = ["<10","10-14","15-19", "20-24", "25-29", "30-34", "35-39", ">40"]

In [9]:
# slice data with pd.ct and add a column for the bins on age groups
purchase_data_df["Age Group"] = pd.cut(purchase_data_df["Age"], bins, labels=age_ranges_labels)
#purchase_data_df.head(10)

In [10]:
# Place the data series into a new column inside of the DataFrame
num_purch_per_group = purchase_data_df.groupby("Age Group")["Purchase ID"].count()
#num_purch_per_group.head(10)

In [11]:
#find the average purchase price per age group
purchase_data_df.groupby("Age Group")["Price"].mean()
avg_purch_price_per_group = purchase_data_df.groupby("Age Group")["Price"].mean()
#avg_purch_price_per_group.head(10)

In [12]:
#find the total purchase price per age group
purchase_data_df.groupby("Age Group")["Price"].sum()
tot_purch_per_group = purchase_data_df.groupby("Age Group")["Price"].sum()
#tot_purch_per_group.head(10)

In [13]:
#find the avg puch total per person per age group

num_unique_people_per_group = purchase_data_df.groupby("Age Group")["SN"].nunique()
#num_unique_people_per_group.head(10)

avg_purch_total_per_person = tot_purch_per_group / num_unique_people_per_group
#avg_purch_total_per_person.head(10)

In [14]:
# Place all of the data found into a summary DataFrame
summary_table = pd.DataFrame({"Purchase Count": num_purch_per_group,
                              "Average Purchase Price": avg_purch_price_per_group,
                              "Total Purchase Value": tot_purch_per_group,
                              "Avg Total Purchase Per Person": avg_purch_total_per_person})

#clean up $ amounts
summary_table["Average Purchase Price"]  = summary_table["Average Purchase Price"].map("${:.2f}".format)
summary_table["Total Purchase Value"] = summary_table["Total Purchase Value"].map("${:.2f}".format)
summary_table["Avg Total Purchase Per Person"] = summary_table["Avg Total Purchase Per Person"].map("${:.2f}".format)

#print it out
summary_table

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
>40,12,$3.04,$36.54,$3.32


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [15]:
#find the purchase count per person
num_purch_by_person = purchase_data_df.groupby("SN")["Purchase ID"].count()
#num_purch_by_person

#find the total purchase value per person
tot_purch_price_per_person = purchase_data_df.groupby("SN")["Price"].sum()
#tot_purch_price_per_person

#find the average purchase amount  per person
avg_purch_price_per_person = purchase_data_df.groupby("SN")["Price"].mean()
#avg_purch_price_per_person

#put in summary df
summary_table_top_spenders = pd.DataFrame({"Purchase Count": num_purch_by_person,
                                  "Average Purchase Price": avg_purch_price_per_person,
                                  "Total Purchase Value": tot_purch_price_per_person})

#sort by total purchase value descending and reset index
biggest_spender_df = summary_table_top_spenders.sort_values(["Total Purchase Value"], ascending=False).head(5)
biggest_spender_df.reset_index(inplace=True)
biggest_spender_df

#clean up $ amounts
biggest_spender_df["Average Purchase Price"] = biggest_spender_df["Average Purchase Price"].map("${:.2f}".format)
biggest_spender_df["Total Purchase Value"] = biggest_spender_df["Total Purchase Value"].map("${:.2f}".format)


#print it out
biggest_spender_df

,SN,Purchase Count,Average Purchase Price,Total Purchase Value
0,Lisosia93,5,$3.79,$18.96
1,Idastidru52,4,$3.86,$15.45
2,Chamjask73,3,$4.61,$13.83
3,Iral74,4,$3.40,$13.62
4,Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [16]:
#Retrieve the Item ID, Item Name, and Item Price columns
organized_df = purchase_data_df[["Item ID", "Item Name", "Price"]]
#rename Price Column to Item Price
organized_df = organized_df.rename(columns={"Price":"Item Price"})
organized_df.head(10)

#Group by Item ID and Item Name. Perform calculations to obtain purchase count, 
#item price, and total purchase value
most_popular_df = organized_df.groupby(["Item ID","Item Name"])
purchase_count = most_popular_df["Item ID"].count()
tot_purch_value = most_popular_df["Item Price"].sum()
item_price =tot_purch_value / purchase_count


#Create a summary data frame to hold the results
summary_most_popular_df = pd.DataFrame({"Purchase Count": purchase_count,
                                        "Item Price": item_price,
                                        "Total Purchase Value": tot_purch_value})
summary_most_popular_df.head(10)

#Sort the purchase count column in descending order
summary_sorted_df = summary_most_popular_df.sort_values(["Purchase Count"], ascending=False).head(5)
summary_sorted_df.reset_index(inplace=True)
summary_sorted_df

#Optional: give the displayed data cleaner formatting
summary_sorted_df["Item Price"] = summary_sorted_df["Item Price"].map("${:.2f}".format)
summary_sorted_df["Total Purchase Value"] = summary_sorted_df["Total Purchase Value"].map("${:.2f}".format)

#Display a preview of the summary data frame
summary_sorted_df

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
0,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
1,145,Fiery Glass Crusader,9,$4.58,$41.22
2,108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
3,82,Nirvana,9,$4.90,$44.10
4,19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [17]:
#Sort the above table by total purchase value in descending order
summ_most_profit_df = summary_most_popular_df.sort_values(["Total Purchase Value"], ascending=False).head(5)
summ_most_profit_df.reset_index(inplace=True)
summ_most_profit_df
summ_most_profit_df.head(5)

#Optional: give the displayed data cleaner formatting
summ_most_profit_df["Item Price"] = summ_most_profit_df["Item Price"].map("${:.2f}".format)
summ_most_profit_df["Total Purchase Value"] = summ_most_profit_df["Total Purchase Value"].map("${:.2f}".format)

#Display a preview of the data frame
summ_most_profit_df

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
0,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
1,82,Nirvana,9,$4.90,$44.10
2,145,Fiery Glass Crusader,9,$4.58,$41.22
3,92,Final Critic,8,$4.88,$39.04
4,103,Singed Scalpel,8,$4.35,$34.80
